# Hello Web 3

Simple python notebook for some eth basics

## Part 1 - Connect to Local Ganache Server

This part shows how to connect to a local ganache node, and how to use the seed phrase associated with the instance to generate the accounts and access the private keys using the same HD scheme as the server.

In [ ]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
print(w3.isConnected())

In [ ]:
 w3.eth.get_block('latest')

In [ ]:
# The accounts associated with the ganache node
w3.eth.accounts

In [ ]:
# starting ganache deterministically with a seed phrase ensures the following seed phrase is used
# for the pre-built accounts and keys
#
# First account address is 0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F
# First private key is  0xcb1a18dff8cfcee16202bf86f1f89f8b3881107b8192cd06836fda9dbc0fde1b
seed_phrase = "drastic alpha short deposit resource future action base gun inner cover net"

In [ ]:
 w3.eth.account.enable_unaudited_hdwallet_features()
owner_account = w3.eth.account.from_mnemonic(seed_phrase, account_path="m/44'/60'/0'/0/0")

In [ ]:
owner_account.address

In [ ]:
owner_account.key

## Part 2 - Call Smart Contract

Adapted from [Smart Contracts in Python A Complete Guide](https://sesamedisk.com/smart-contracts-in-python-complete-guide/)

In [ ]:
from eth_utils import address
from web3 import Web3
import os
from solcx import compile_standard, install_solc
from dotenv import load_dotenv
import json

In [ ]:
with open("./Token.sol", "r") as file:
    token_file = file.read()

In [ ]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"Token.sol": {"content": token_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    #solc_version="0.8.17+commit.8df45f5f.Linux.g++",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [ ]:
bytecode = compiled_sol["contracts"]["Token.sol"]["Token"]["evm"][
    "bytecode"
]["object"]

In [ ]:
abi = json.loads(
    compiled_sol["contracts"]["Token.sol"]["Token"]["metadata"]
)["output"]["abi"]

In [ ]:
chain_id = 1337

In [ ]:
 nonce = w3.eth.get_transaction_count(owner_account.address)

In [ ]:
# Deploy the contract by invoking the constructor

# initialize contract
Token = w3.eth.contract(abi=abi, bytecode=bytecode)
nonce = w3.eth.getTransactionCount(owner_account.address)
# set up transaction from constructor which executes when firstly
transaction = Token.constructor().buildTransaction(
    {"chainId": chain_id, "from": owner_account.address, "nonce": nonce}
)


In [ ]:
transaction

In [ ]:
signed_tx = w3.eth.signTransaction(transaction)

In [ ]:
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [ ]:
# Deployed from the hardhat getting started sample
contract_address = '0xdb98a5bFBA239000213813B2615B8A96E950A79B'

In [ ]:
token_abi='''
[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":true,"internalType":"address","name":"_to","type":"address"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[],"stateMutability":"nonpayable","type":"function"}]
'''

In [ ]:
token_contract = w3.eth.contract(contract_address,abi=abi)

In [ ]:
token_contract.abi

In [ ]:
 nonce = w3.eth.get_transaction_count(owner_account.address)

In [ ]:
nonce

In [ ]:
token_contract.functions.totalSupply().call()

In [ ]:
token_contract.functions.balanceOf(owner_account.address).call()

In [ ]:
token_contract.functions.balanceOf(w3.eth.accounts[1]).call()

In [ ]:
tx_hash = token_contract.functions.transfer(w3.eth.accounts[1], 200).transact({'from': owner_account.address})

In [ ]:
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [ ]:
tx_receipt

In [ ]:
token_contract.functions.balanceOf(w3.eth.accounts[1]).call()